In [3]:
import json
import random


data = json.load(open('../dataset/Jurassic_Ultra/gender/raw.json'))
anno_data = []
for k, v in data.items():
    
    responses = list(zip(range(len(v['other_attributes'])), v['other_attributes']))
    chosen_pairs = []
    for _ in range(5):
        r1, r2 = random.choices(responses, k=2)
        if r1[0] != r2[0]:
            if (r1[0], r2[0]) not in chosen_pairs and (r2[0], r1[0]) not in chosen_pairs:
                chosen_pairs.append((r1[0], r2[0]))
                ex = {}
                ex['group_id'] = k
                ex['base_question'] = v['base_question']
                ex['r1'] = r1[1]
                ex['r2'] = r2[1]
                anno_data.append(ex)
print(f'Total data size: {len(anno_data)}')

Total data size: 194


In [13]:
n_annotator = 3
n_per_annotator = len(anno_data) // n_annotator


for i in range(n_annotator):
    data = anno_data[i*(n_per_annotator): (i+1)*n_per_annotator]
    print(len(data))
    filename = 'anno_data_%02d.json' % (i+1)
    json.dump(data, open(filename, 'w'), indent=2)


64
64
64


In [15]:
import difflib
import re

def highlight_similarities(text1, text2):
    # Split the texts into words
    words1 = re.findall(r'\S+|\s+', text1)
    words2 = re.findall(r'\S+|\s+', text2)

    # Use difflib to find matching sequences
    matcher = difflib.SequenceMatcher(None, words1, words2)
    
    # Prepare the output strings
    output1 = []
    output2 = []
    
    # Colors for highlighting (using HTML span tags)
    colors = [
        '<span style="background-color: #FFFF00;">',  # Yellow
        '<span style="background-color: #90EE90;">',  # Light Green
        '<span style="background-color: #ADD8E6;">',  # Light Blue
        '<span style="background-color: #FFB6C1;">',  # Light Pink
        '<span style="background-color: #E6E6FA;">',  # Lavender
    ]
    color_index = 0

    for op, i1, i2, j1, j2 in matcher.get_opcodes():
        if op == 'equal':
            # Highlight matching parts with the same color
            color = colors[color_index % len(colors)]
            color_index += 1
            
            segment = ''.join(words1[i1:i2])
            output1.append(f"{color}{segment}</span>")
            output2.append(f"{color}{segment}</span>")
        else:
            # Leave non-matching parts as they are
            output1.extend(words1[i1:i2])
            output2.extend(words2[j1:j2])
    
    # Join the output words back into strings
    return ''.join(output1), ''.join(output2)

# Example usage
text1 = "The quick brown fox\n jumps over the lazy dog.".split()
text2 = "The quick red fox leaps over the sleepy cat.".split()

highlighted1, highlighted2 = highlight_similarities(text1, text2)
print("Text 1 (highlighted):", highlighted1)
print("Text 2 (highlighted):", highlighted2)

Text 1 (highlighted): <span style="background-color: #FFFF00;">The quick </span>brown<span style="background-color: #90EE90;"> fox</span>
 jumps<span style="background-color: #ADD8E6;"> over the </span>lazy<span style="background-color: #FFB6C1;"> </span>dog.
Text 2 (highlighted): <span style="background-color: #FFFF00;">The quick </span>red<span style="background-color: #90EE90;"> fox</span> leaps<span style="background-color: #ADD8E6;"> over the </span>sleepy<span style="background-color: #FFB6C1;"> </span>cat.


In [17]:
matcher = difflib.SequenceMatcher(None, text1, text2)
for b in matcher.get_matching_blocks():
    print(text1[b.a:b.a + b.size])

The quick 
r
 fox
 
ps over the 
l
y 
.

